In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class TSNE:
    def __init__(self,k_dim=2, perplexity=3, momentum=0.1 ,lr=300,
                 max_iter_sigma=1000,max_iter_gd=1000):
        self.perplexity = perplexity
        self.momentum = momentum
        self.lr = lr
        self.max_iter_sigma = max_iter_sigma
        self.k_dim = k_dim
        self.max_iter_gd = max_iter_gd
        
    def pdf(self,X,sigma,j,i):
        return np.exp(-np.linalg.norm(X[i]-X[j])**2/(2*sigma**2))
    
    def calc_p_j_i(self,X,sigma,j,i):
        nominator = self.pdf(X,sigma,j,i)
        denominator=0
        for k in range(X.shape[0]):
            if k!=i:
                denominator+=self.pdf(X,sigma,k,i)
        return nominator/denominator
        
    def get_sigma_point(self,X, i):
        sigma = 500
        tol = 0.001
        diff = sigma/2
        for _ in range(self.max_iter_sigma):
            H=0
            for j in range(X.shape[0]):
                if i!=j:
                    p_j_i=self.calc_p_j_i(X,sigma,j,i)
                    H-=p_j_i*np.log2(p_j_i)
            pred_prep=2**H
            
            if np.abs(pred_prep - self.perplexity) <= tol:
                break
            if pred_prep > self.perplexity:
                sigma -= diff
            else:
                sigma += diff
            diff /=2
        return sigma
        
    def get_sigma_all(self,X):
        sigmas = []
        for i in range(X.shape[0]):
            sigma=self.get_sigma_point(X, i)
            sigmas.append(sigma)
        return sigmas
    
    def fit_transform(self,X):
        sigmas = self.get_sigma_all(X)
        y = np.random.normal(0,0.0001,(X.shape[0],self.k_dim))
        p_matrix = self.calc_p_matrix(X,sigmas)
        for _ in range(self.max_iter_gd):
            q_matrix = self.calc_q_matrix(y)
            
            y = y - self.lr*self.compute_gradient(y,p_matrix,q_matrix)
           
        return y
                             
    def calc_p_matrix(self,X,sigmas):
        p_matrix = np.zeros((X.shape[0],X.shape[0]))
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                if i!=j:
                    pji=self.calc_p_j_i(X,sigmas[i],j,i)
                    pij=self.calc_p_j_i(X,sigmas[j],i,j)
                    p_matrix[i][j] = (pji + pij)/(2*X.shape[0])
                    p_matrix[j][i] = p_matrix[i][j]
        np.fill_diagonal(p_matrix,1)
        return p_matrix
    
    def t_dist(self,y,k,l):
        return 1/(1+(np.linalg.norm(y[k]-y[l])**2))
        
    def calc_q_j_i(self,y,j,i):
        nominator = self.t_dist(y,j,i)
        denominator=0
        for k in range(y.shape[0]):
            for l in range(y.shape[0]):
                if k!=l:
                    denominator+=self.t_dist(y,k,l)
        return nominator/denominator
    
    def calc_q_matrix(self,y):
        q_matrix = np.zeros((y.shape[0],y.shape[0]))
        for i in range(y.shape[0]):
            for j in range(y.shape[0]):
                if i!=j:
                    q_matrix[j][i]=self.calc_q_j_i(y,j,i)
                    q_matrix[i][j] = q_matrix[i][j]
        np.fill_diagonal(q_matrix,1)
        return q_matrix
    
    def compute_gradient(self,y,p,q):
        partials = []
        for i in range(y.shape[0]):
            derivative =0
            for j in range(y.shape[0]):
                derivative +=4*(p[i][j]-q[i][j])*(y[i]-y[j])*self.t_dist(y,i,j)
            partials.append(derivative)
        return np.array(partials)
    